In [2]:
import glob, os, sys
from pprint import pprint
import pickle
data_dir = 'data_files/'
### Let's first get a list of files from the datafiles directory that I may (or may not?? ) want to parse
dataFiles = glob.glob(data_dir+'20160*')

In [3]:
print dataFiles[20]

data_files/20160402


In [4]:
#!pip install plotly
import plotly
from plotly.graph_objs import Scatter, Layout, Bar
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [7]:
executed_jobs = 0
LogOutput = []
for f in dataFiles:
    #print f
    LogOutput.append( analyze_data_file( f) )

In [ ]:
def computeAWSCosts()

In [ ]:
##  https://aws.amazon.com/ec2/pricing/on-demand/
## On Demand versus spot pricing is one key factor
### MATLAB?

###m4.xlarge	4	13	16	EBS Only	$0.239 per Hour
## for now assume m4 largeFor 
#https://pricing.us-east-1.amazonaws.com/offers/v1.0/aws/{offer_code}/current/index.{format}
AWSPriceIndex = {
  "formatVersion" : "v1.0",
  "disclaimer" : "This pricing list is for informational purposes only. All prices are subject to the additional terms included in the pricing pages on http://aws.amazon.com. All Free Tier prices are also subject to the terms included at https://aws.amazon.com/free/",
  "publicationDate" : "2016-10-26T20:54:55Z",
  "offers" : {
    "AmazonS3" : {
      "offerCode" : "AmazonS3",
      "versionIndexUrl" : "/offers/v1.0/aws/AmazonS3/index.json",
      "currentVersionUrl" : "/offers/v1.0/aws/AmazonS3/current/index.json"
    },
    "AmazonGlacier" : {
      "offerCode" : "AmazonGlacier",
      "versionIndexUrl" : "/offers/v1.0/aws/AmazonGlacier/index.json",
      "currentVersionUrl" : "/offers/v1.0/aws/AmazonGlacier/current/index.json"
    },
    "AmazonSES" : {
      "offerCode" : "AmazonSES",
      "versionIndexUrl" : "/offers/v1.0/aws/AmazonSES/index.json",
      "currentVersionUrl" : "/offers/v1.0/aws/AmazonSES/current/index.json"
    },
    "AmazonRDS" : {
      "offerCode" : "AmazonRDS",
      "versionIndexUrl" : "/offers/v1.0/aws/AmazonRDS/index.json",
      "currentVersionUrl" : "/offers/v1.0/aws/AmazonRDS/current/index.json"
    },
    "AmazonSimpleDB" : {
      "offerCode" : "AmazonSimpleDB",
      "versionIndexUrl" : "/offers/v1.0/aws/AmazonSimpleDB/index.json",
      "currentVersionUrl" : "/offers/v1.0/aws/AmazonSimpleDB/current/index.json"
    },
    "AmazonDynamoDB" : {
      "offerCode" : "AmazonDynamoDB",
      "versionIndexUrl" : "/offers/v1.0/aws/AmazonDynamoDB/index.json",
      "currentVersionUrl" : "/offers/v1.0/aws/AmazonDynamoDB/current/index.json"
    },
    "AmazonEC2" : {
      "offerCode" : "AmazonEC2",
      "versionIndexUrl" : "/offers/v1.0/aws/AmazonEC2/index.json",
      "currentVersionUrl" : "/offers/v1.0/aws/AmazonEC2/current/index.json"
    },
    "AmazonRoute53" : {
      "offerCode" : "AmazonRoute53",
      "versionIndexUrl" : "/offers/v1.0/aws/AmazonRoute53/index.json",
      "currentVersionUrl" : "/offers/v1.0/aws/AmazonRoute53/current/index.json"
    },
    "AmazonRedshift" : {
      "offerCode" : "AmazonRedshift",
      "versionIndexUrl" : "/offers/v1.0/aws/AmazonRedshift/index.json",
      "currentVersionUrl" : "/offers/v1.0/aws/AmazonRedshift/current/index.json"
    },
    "AmazonElastiCache" : {
      "offerCode" : "AmazonElastiCache",
      "versionIndexUrl" : "/offers/v1.0/aws/AmazonElastiCache/index.json",
      "currentVersionUrl" : "/offers/v1.0/aws/AmazonElastiCache/current/index.json"
    },
    "AmazonCloudFront" : {
      "offerCode" : "AmazonCloudFront",
      "versionIndexUrl" : "/offers/v1.0/aws/AmazonCloudFront/index.json",
      "currentVersionUrl" : "/offers/v1.0/aws/AmazonCloudFront/current/index.json"
    },
    "awskms" : {
      "offerCode" : "awskms",
      "versionIndexUrl" : "/offers/v1.0/aws/awskms/index.json",
      "currentVersionUrl" : "/offers/v1.0/aws/awskms/current/index.json"
    },
    "AmazonVPC" : {
      "offerCode" : "AmazonVPC",
      "versionIndexUrl" : "/offers/v1.0/aws/AmazonVPC/index.json",
      "currentVersionUrl" : "/offers/v1.0/aws/AmazonVPC/current/index.json"
    }
  }
}

In [10]:
PricingAPIUrl = 'https://pricing.us-east-1.amazonaws.com/offers/v1.0/aws/AmazonEC2/current/index.json'
import urllib, json

response = urllib.urlopen(PricingAPIUrl)
print response
Pricedata = json.loads(response.read())


<addinfourl at 4587717624 whose fp = <socket._fileobject object at 0x126a91e50>>
[u'offerCode', u'terms', u'formatVersion', u'version', u'products', u'publicationDate', u'disclaimer']


In [34]:
### need to find the instance to use as a sample... need to search this mess
##"location" : "US East (N. Virginia)",
from pprint import pprint
## Instance types vary dramatically.. will get Rajan to make a web widget from this..

exampsPrinted = 0

locCode = "US East (N. Virginia)"
for sku,ec2Item in Pricedata['products'].iteritems():
    try:
        if locCode in ec2Item['attributes']['location']:

            if 'Compute Instance' in ec2Item['productFamily']:
                if 'Linux' in ec2Item['attributes']['operatingSystem']:
#         if ( 'productFamily' in ec2Item['attributes']):
#             if (ec2Item['attributes']['productFamily']  == 'Compute Instance'):
# #         and 'Compute Instance' in ec2Item['attributes']['productFamily']:
                    pprint(ec2Item)
                    exampsPrinted +=1
    except:
        pass
#        print "Weird key?"
#        pprint( ec2Item)
## So this doesn't even list the prices!?

## NVM the SKU's are in a separate key... so this needs to go into mongo..
    if exampsPrinted > 10:
        sys.exit()

{u'attributes': {u'clockSpeed': u'2.9 GHz',
                 u'currentGeneration': u'Yes',
                 u'dedicatedEbsThroughput': u'1000 Mbps',
                 u'enhancedNetworkingSupported': u'Yes',
                 u'instanceFamily': u'Compute optimized',
                 u'instanceType': u'c4.2xlarge',
                 u'licenseModel': u'No License required',
                 u'location': u'US East (N. Virginia)',
                 u'locationType': u'AWS Region',
                 u'memory': u'15 GiB',
                 u'networkPerformance': u'High',
                 u'operatingSystem': u'Linux',
                 u'operation': u'RunInstances',
                 u'physicalProcessor': u'Intel Xeon E5-2666 v3 (Haswell)',
                 u'preInstalledSw': u'NA',
                 u'processorArchitecture': u'64-bit',
                 u'processorFeatures': u'Intel AVX; Intel AVX2; Intel Turbo',
                 u'servicecode': u'AmazonEC2',
                 u'storage': u'EBS only',
  

SystemExit: 

To exit: use 'exit', 'quit', or Ctrl-D.


In [32]:
%tb

SystemExit: 

In [36]:
#print len(allExecutedJobs)  ### Only 30,000 jobs apparently were executed.. I must have deleted a lot of them?
#pprint( allExecutedJobs[2]) ### Just print the second record
import datetime as dt


totalWallTime = dt.timedelta(hours=0, minutes=0)
totalCpuTime =  dt.timedelta(hours=0, minutes=0)

for idx,row in enumerate(allExecutedJobs):
    ruWt = row['resourcesUsedWallTime']  
    #print row

    ruWtDt  = dt.datetime.strptime(ruWt,"%H:%M:%S")
    curJobWallTime = dt.timedelta(hours=ruWtDt.hour, minutes=ruWtDt.minute, seconds=ruWtDt.second)
    totalWallTime += curJobWallTime
    
    ###Lets do the same thing for CPUTime
    ruCpu = row['cpuTotal']
    ruCpuDt  = dt.datetime.strptime(ruCpu,"%H:%M:%S")
    curJobCpuTime = dt.timedelta(hours=ruCpuDt.hour, minutes=ruCpuDt.minute, seconds=ruCpuDt.second)
    totalCpuTime += curJobCpuTime
    #'cpuTotal': '02:27:06'
    


print totalWallTime
print totalCpuTime

1363 days, 6:54:31
1759 days, 16:28:03


In [45]:
avgJobTime  = totalWallTime/15040

In [52]:
avgJobTime.total_seconds()
## So it's about 3 hours per job
# we allocate 24GB per job with 4 cores..
#Closest match is 32 GB 4 core
### Let's assume we use a memory optimized instance r3.xlarge	4	13	30.5	1 x 80 SSD	$0.333 per Hour
## Has 4 cores


CostPerHour = 0.333
JobLengthInHours = 3
NumJobs = 15000


TotalComputeCost = CostPerHour * JobLengthInHours * NumJobs
print TotalComputeCost

StoragePerGB = 0.03
EstimatedStorage  = 40 ## in terabytes
## This is per month BTW..

###$0.0300 per GB
StorageCostsPerMonth = StoragePerGB * EstimatedStorage * 1000 #to TB from GB
print StorageCostsPerMonth

14985.0
1200.0


In [44]:
### Had 15040 jobs... let's compute the average length per job
print totalWallTime
## I am ignoring transfer costs into/out of AWS

1363 days, 6:54:31


In [ ]:

pprint(LogOutput)

In [11]:
import plotly.plotly as py
import plotly.graph_objs as go

xdata = [ x['LogDate'].replace('data_files',' ')  for x in LogOutput]
ydata = [y['executed_jobs'] for y in LogOutput]

data = [go.Bar( x=xdata, y=ydata)]
iplot(data, filename='basic-bar')

In [6]:
import re

mfrSubmit=  re.compile(' .*user=(?P<user>.*) group=(?P<group>.*) jobname=(?P<jobname>.*) queue=(?P<queue>.*) ctime=(?P<ctime>\d+) qtime=(?P<qtime>\d+) \
etime=(?P<etime>\d+) start=(?P<start>\d+) owner=(?P<owner>.*) exec_host=(?P<exec_host>.*)' )
##my first regex

mfrExec = re.compile('.*;(?P<jobEntryType>.);.*user=(?P<user>.*) group=(?P<group>.*) jobname=(?P<jobname>.*) queue=(?P<queue>.*) ctime=(?P<ctime>\d+) qtime=(?P<qtime>\d+) \
etime=(?P<etime>\d+) start=(?P<start>\d+) owner=(?P<owner>.*) exec_host=(?P<exec_host>.*)' )


mfrExec2 = re.compile('.*;(?P<jobEntryType>.);.*user=(?P<user>.*) group=(?P<group>.*) jobname=(?P<jobname>.*) queue=(?P<queue>.*) ctime=(?P<ctime>\d+) qtime=(?P<qtime>\d+) etime=(?P<etime>\d+) start=(?P<start>\d+) owner=(?P<owner>.*) exec_host=(?P<exec_host>.*) Resource_List.neednodes=(?P<needNodes>\d+):ppn=(?P<needppn>\d+):(?P<needResourceName>.*) Resource_List.walltime=(?P<ndwalltime>.*) session=(?P<session>\d+) total_execution_slots=(?P<totExecSlots>\d+) unique_node_count=(?P<uniqueNodeCount>\d+) end=(?P<endtime>\d+) Exit_status=(?P<exitStatus>.) resources_used.cput=(?P<cpuTotal>.*) resources_used.mem=(?P<resourcesUsedMem>\d+kb) resources_used.vmem=(?P<resourcesedUsedVMem>\d+kb) resources_used.walltime=(?P<resourcesUsedWallTime>.*)')


allExecutedJobs = []

def analyze_data_file( fileName):
    """This will scan a log file from PBS/Torque and extract meaningful information about how many jobs were
    run, how long they took, etc"""

    executed_jobs = 0
    wall_time = 0 ### This really should be parsed... and probably load the data into a database
    
    with open(fileName, 'rU') as infile:
        for line in infile:
            line = line.strip()
            m = mfrExec2.search(line)
            if m:
                if m.groupdict()['jobEntryType'] == "E":
                    curJobInfo = m.groupdict()
                    executed_jobs +=1
                    allExecutedJobs.append(curJobInfo)

                    
                    
    return (  {'LogDate': fileName, 'executed_jobs': executed_jobs})
            

In [5]:
### Let's do some VERY basic analysis...

In [11]:
with open('allExecJobs.pickle', 'w') as fp:
    pickle.dump(allExecutedJobs,fp)

In [ ]:
sample_file = dataFiles[3]
with  open(sample_file,'rU') as fp:
    sampleData = fp.readlines()

In [ ]:
from pprint import pprint


mfr=  re.compile(' .*user=(?P<user>.*) group=(?P<group>.*) jobname=(?P<jobname>.*) queue=(?P<queue>.*) ctime=(?P<ctime>\d+) qtime=(?P<qtime>\d+) \
etime=(?P<etime>\d+) start=(?P<start>\d+) owner=(?P<owner>.*) exec_host=(?P<exec_host>.*) Resource_List.(?P<rsrc>.*) Resource_List.(?P<rsrc2>.*) \
Resource_List.(?P<rsrc3>.*)' )


mfr=  re.compile(' .*user=(?P<user>.*) group=(?P<group>.*) jobname=(?P<jobname>.*) queue=(?P<queue>.*) ctime=(?P<ctime>\d+) qtime=(?P<qtime>\d+) \
etime=(?P<etime>\d+) start=(?P<start>\d+) owner=(?P<owner>.*) exec_host=(?P<exec_host>.*) Resource_List.(?P<rsrc>.*) Resource_List.(?P<rsrc2>.*) \
Resource_List.(?P<rsrc3>.*)' )


###session=1417 total_execution_slots=2 unique_node_count=1 end=1458100531 Exit_status=0 resources_used.cput=00:00:17 resources_used.mem=2703476kb resources_used.vmem=4724852kb resources_used.walltime=00:01:08
### I can parse the above separtely

rsrcUsedList = re.compile('session=(?P<sessionID>.*) total_execution_slots=(?P<tExecSlots>\d+) unique_node_count=(?P<uNodeCount>\d+) \
end=(?P<end>\d+) Exit_status=(?P<ExitStatus>.*) resources_used.cput=(?P<cput>.*) resources_used.mem=(?P<memUsed>\d+kb) \
resources_used.vmem=(?P<vmem>\d+kb) resources_used.walltime=(?P<walltime>.*)')

sid = 5

print sampleData[sid]
# m= mfr.search(sampleData[sid])
# if m:
#     pprint( m.groupdict())

m2 = rsrcUsedList.search(sampleData[sid])
if m2:
    pprint (m2.groupdict())
                          

In [1]:
for r in sampleData:
    m2 = rsrcUsedList.search(r)
    if m2:
        pprint(m2.groupdict())

NameError: name 'sampleData' is not defined